
# Remove duplicate PMids

In [ ]:
import pandas as pd

df = pd.read_excel('data_all.xlsx', sheet_name='article')  
GMDB = pd.read_csv('image_metadata_v1.1.0.tsv', sep='\t')
GMDB = GMDB.dropna(subset=['pmid'])
GMDB['pmid'] = pd.to_numeric(GMDB['pmid'], errors='coerce').astype('Int64')  # 注意大写的Int64
df['pmid'] = pd.to_numeric(df['pmid'], errors='coerce').astype('Int64')

merged = GMDB.merge(df[['pmid']], on='pmid', how='left', indicator=True)
GMDB_unique = merged[merged['_merge'] == 'left_only'].drop('_merge', axis=1)

GMDB_unique = GMDB[~GMDB['pmid'].isin(df['pmid']) & GMDB['pmid'].notna()]
print(f"GMDB原始记录数: {len(GMDB)}")
print(f"唯一记录数: {len(GMDB_unique)}")
GMDB_unique.to_csv("GMDB.csv", index=False)

# Bind the OMIM id

In [ ]:
df = pd.read_excel('data_all.xlsx', sheet_name='article')  
GMDB = pd.read_csv('GMDB.csv')
GMDB_syndrome = pd.read_csv('gmdb_syndromes_v1.1.0.tsv', sep='\t')
GMDB['OMIM'] = GMDB['internal_syndrome_name'].map(GMDB_syndrome.set_index('syndrome_name')['OMIM'])

GMDB = GMDB.dropna(subset=['OMIM'])
GMDB.to_csv("GMDB.csv", index=False)

# Create nodes

In [ ]:
# article node
from Bio import Entrez
from Bio.Entrez import efetch, read
import time
from tqdm import tqdm
article_node = GMDB[["pmid", "author"]]
article_node = article_node.drop_duplicates()

def fetch_pubmed_details(pmids, batch_size=100):
    all_details = []
    # 将 PMID 分批次处理
    for i in tqdm(range(0, len(pmids), batch_size)):
        batch = pmids[i:i+batch_size]
        try:
            handle = efetch(db="pubmed", id=batch, retmode="xml")
            records = read(handle)["PubmedArticle"]
            for record in records:
                article = record["MedlineCitation"]["Article"]
                pmid = record["MedlineCitation"]["PMID"]
                title = article.get("ArticleTitle", "N/A")
                journal_info = article.get("Journal", {})
                journal_name = journal_info.get("Title", "N/A")
                all_details.append({
                    "PMID": pmid,
                    "Title": title,
                    "magazineName": journal_name,
                })
                
            # 遵守 NCBI 的请求频率限制（每秒1次）
            time.sleep(1)  # 无 API Key 时需更严格限制
        except Exception as e:
            print(f"Error fetching PMIDs {batch}: {str(e)}")
            continue
    return all_details

pmids = article_node["pmid"].tolist()
pubmed_data = fetch_pubmed_details(pmids)
pubmed_data_df = pd.DataFrame(pubmed_data)
pubmed_data_df['PMID'] = pd.to_numeric(pubmed_data_df['PMID'], errors='coerce').astype('Int64')
article_node = pd.merge(
    article_node,
    pubmed_data_df,
    left_on="pmid",
    right_on="PMID",
    how="left"
).drop("PMID", axis=1)

article_node = article_node.rename(columns={"Title": "articleName",})

In [24]:
# disease node
disease_node = GMDB[["OMIM", "internal_syndrome_name"]]
disease_node = disease_node.drop_duplicates()
disease_node.columns = ["did", "dname"]
disease_node['did'] = pd.to_numeric(disease_node['did'], errors='coerce').astype('Int64')
    # merge
df_disease = pd.read_excel('data_all.xlsx', sheet_name='diseases')
df_disease = df_disease.rename(columns={"Did": "did",})
merged = disease_node.merge(df_disease[['did']], on='did', how='left', indicator=True)
disease_node = merged[merged['_merge'] == 'left_only'].drop('_merge', axis=1)

In [25]:
# phenotype node
import pronto
phenotype_node = GMDB[["present_features"]]
phenotype_node = phenotype_node.rename(columns={"present_features": "pid",})
phenotype_node = phenotype_node.dropna(subset=['pid'])
    # expand
phenotype_node["pid"] = phenotype_node["pid"].str.split(";").apply(lambda x: [s.strip() for s in x])
phenotype_node = phenotype_node.explode("pid").reset_index(drop=True)
phenotype_node = phenotype_node.drop_duplicates()
    # merge
df_phenotype = pd.read_excel('data_all.xlsx', sheet_name='phenotype')
df_phenotype = df_phenotype.rename(columns={"Pid": "pid",})
merged = phenotype_node.merge(df_phenotype[['pid']], on='pid', how='left', indicator=True)
phenotype_node = merged[merged['_merge'] == 'left_only'].drop('_merge', axis=1)
    # mapping name
hpo = pronto.Ontology("hp.obo")
def get_hpo_term(hpo_id):
    term = hpo.get(hpo_id)
    return term.name if term else "Unknown"
phenotype_node["phenotypeName"] = phenotype_node["pid"].apply(get_hpo_term)

C:\Users\15085\AppData\Local\Temp\ipykernel_26532\2014670767.py:16: UnicodeWarning: unsound encoding, assuming ISO-8859-1 (73% confidence)
  hpo = pronto.Ontology("hp.obo")


In [26]:
# genotype node
genotype_node = GMDB[["gene_names","gene_entrez_ids"]]
genotype_node = genotype_node.rename(columns={"gene_names": "geneName",  "gene_entrez_ids": "geneId"})
genotype_node = genotype_node.dropna(subset=['geneName'])
    # expand
genotype_node["geneName"] = genotype_node["geneName"].str.split(r",\s*")  # 按逗号分割，允许空格
genotype_node["geneId"] = genotype_node["geneId"].str.split(r",\s*")
genotype_node = genotype_node.explode(["geneName", "geneId"]).reset_index(drop=True)
genotype_node = genotype_node.drop_duplicates()
genotype_node["geneId"] = pd.to_numeric(genotype_node["geneId"]).astype('Int64')
    # merge
df_genotype = pd.read_excel('data_all.xlsx', sheet_name='genotype')
df_genotype = df_genotype.rename(columns={"gene_id": "geneId",})
df_genotype["geneId"] = pd.to_numeric(df_genotype["geneId"], errors='coerce').astype('Int64')
merged = genotype_node.merge(df_genotype[['geneId']], on='geneId', how='left', indicator=True)
genotype_node = merged[merged['_merge'] == 'left_only'].drop('_merge', axis=1)

In [27]:
# variation node (details, category)
from ast import literal_eval
variation_node = GMDB[["hgvs"]]
variation_node = variation_node.rename(columns={"hgvs": "details",})
variation_node["details"] = variation_node["details"].apply(lambda x: literal_eval(x) if isinstance(x, str) else x)
variation_node = variation_node[variation_node["details"].apply(len) > 0]
variation_node["details"] = variation_node["details"].apply(lambda x: ", ".join(map(str, x)) if isinstance(x, list) else str(x))
variation_node = variation_node.drop_duplicates()
    # merge
df_variation = pd.read_excel('data_all.xlsx', sheet_name='relation_gd')
merged = variation_node.merge(df_variation[['details']], on='details', how='left', indicator=True)
variation_node = merged[merged['_merge'] == 'left_only'].drop('_merge', axis=1)

In [28]:
# sample node (sid, gender, race, samplenumber, age)
sample_node = GMDB[["pmid", "gender", "ethnicity_category", "age_year", "age_month"]]
sample_node = sample_node.rename(columns={"pmid": "sid", "ethnicity_category":"race", })
sample_node["sid"] = "s_" + sample_node["sid"].astype(str) + "_" + (sample_node.groupby("sid").cumcount() + 1).astype(str)
sample_node["gender"] = sample_node["gender"].replace({"female": "F", "male": "M"})
sample_node["age"] = sample_node["age_year"] + sample_node["age_month"] / 12
sample_node = sample_node.drop(columns=["age_year", "age_month"])
sample_node = sample_node.rename(columns={"age": "year", })

In [29]:
article_node.to_csv("./nodes/article_node.csv", index=False)
disease_node.to_csv("./nodes/disease_node.csv", index=False)
phenotype_node.to_csv("./nodes/phenotype_node.csv", index=False)
genotype_node.to_csv("./nodes/genotype_node.csv", index=False)
variation_node.to_csv("./nodes/variation_node.csv", index=False)
sample_node.to_csv("./nodes/sample_node.csv", index=False)

# Cyphers to import nodes
LOAD CSV WITH HEADERS FROM 'file:///article_node.csv' AS row CREATE (:Article { pmid: toInteger(row.pmid), author: row.author, articleName: row.articleName, magazineName: row.magazineName }); 
LOAD CSV WITH HEADERS FROM 'file:///disease_node.csv' AS row CREATE (:Disease { did: toInteger(row.did), dname: row.dname }); 
LOAD CSV WITH HEADERS FROM 'file:///phenotype_node.csv' AS row CREATE (:FacePhenotype { pid: row.pid, phenotypeName: row.phenotypeName }); 
LOAD CSV WITH HEADERS FROM 'file:///genotype_node.csv' AS row CREATE (:Genotype { geneName: row.geneName, geneId: toInteger(row.geneId) }); 
LOAD CSV WITH HEADERS FROM 'file:///variation_node.csv' AS row CREATE (:Variation { details: row.details}); 
LOAD CSV WITH HEADERS FROM 'file:///sample_node.csv' AS row CREATE (:Sample {sid: row.sid, gender: row.gender, race: row.race, year: row.year }); 

# Create relationships

In [30]:
# relation_sample_phenotype   (sid, pid)
relation_sample_phenotype = GMDB[["present_features", "pmid"]]
relation_sample_phenotype = relation_sample_phenotype.rename(columns={"pmid": "sid", "present_features":"pid", })
relation_sample_phenotype["sid"] = "s_" + relation_sample_phenotype["sid"].astype(str) + "_" + (relation_sample_phenotype.groupby("sid").cumcount() + 1).astype(str)
relation_sample_phenotype = relation_sample_phenotype.dropna(subset=['pid'])
    # expand
relation_sample_phenotype["pid"] = relation_sample_phenotype["pid"].str.split(";").apply(lambda x: [s.strip() for s in x])
relation_sample_phenotype = relation_sample_phenotype.explode("pid").reset_index(drop=True)
relation_sample_phenotype = relation_sample_phenotype.drop_duplicates()
relation_sample_phenotype['type'] = "Mention_FP"

In [31]:
# relation_sample_variation  (sid, details)
relation_sample_variation = GMDB[["pmid", "hgvs"]]
relation_sample_variation = relation_sample_variation.rename(columns={"pmid": "sid", "hgvs": "details",})
relation_sample_variation["sid"] = "s_" + relation_sample_variation["sid"].astype(str) + "_" + (relation_sample_variation.groupby("sid").cumcount() + 1).astype(str)
relation_sample_variation["details"] = relation_sample_variation["details"].apply(lambda x: literal_eval(x) if isinstance(x, str) else x)
relation_sample_variation = relation_sample_variation[relation_sample_variation["details"].apply(len) > 0]

relation_sample_variation["details"] = relation_sample_variation["details"].apply(lambda x: ", ".join(map(str, x)) if isinstance(x, list) else str(x))
relation_sample_variation = relation_sample_variation.drop_duplicates()
relation_sample_variation['type'] = "Mention_Var"

In [32]:
# relation_variation_gene (details, gid)
relation_variation_gene = GMDB[["hgvs", "gene_entrez_ids"]]
relation_variation_gene = relation_variation_gene.rename(columns={"hgvs": "details", "gene_entrez_ids": "geneId",})
relation_variation_gene = relation_variation_gene.dropna(subset=['geneId'])
relation_variation_gene["details"] = relation_variation_gene["details"].apply(lambda x: literal_eval(x) if isinstance(x, str) else x)
relation_variation_gene = relation_variation_gene[relation_variation_gene["details"].apply(len) > 0]
relation_variation_gene["details"] = relation_variation_gene["details"].apply(lambda x: ", ".join(map(str, x)) if isinstance(x, list) else str(x))
    # expand
relation_variation_gene["geneId"] = relation_variation_gene["geneId"].str.split(r",\s*")
relation_variation_gene = relation_variation_gene.explode(["geneId"]).reset_index(drop=True)
relation_variation_gene = relation_variation_gene.drop_duplicates()
relation_variation_gene["geneId"] = pd.to_numeric(relation_variation_gene["geneId"]).astype('Int64')
relation_variation_gene['type'] = "Have"

In [33]:
# relation_variation_disease  (details, did)
relation_variation_disease = GMDB[["hgvs", "OMIM"]]
relation_variation_disease = relation_variation_disease.rename(columns={"hgvs": "details", "OMIM": "did",})
relation_variation_disease["details"] = relation_variation_disease["details"].apply(lambda x: literal_eval(x) if isinstance(x, str) else x)
relation_variation_disease = relation_variation_disease[relation_variation_disease["details"].apply(len) > 0]
relation_variation_disease["details"] = relation_variation_disease["details"].apply(lambda x: ", ".join(map(str, x)) if isinstance(x, list) else str(x))
relation_variation_disease['did'] = pd.to_numeric(relation_variation_disease['did'], errors='coerce').astype('Int64')
relation_variation_disease = relation_variation_disease.drop_duplicates()
relation_variation_disease['type'] = "Cause"

In [34]:
# relation_gene_phenotype  (gid, pid)
relation_gene_phenotype = GMDB[["gene_entrez_ids", "present_features"]]
relation_gene_phenotype = relation_gene_phenotype.rename(columns={"gene_entrez_ids": "geneId", "present_features":"pid", })
relation_gene_phenotype = relation_gene_phenotype.dropna(subset=['geneId', 'pid'])
    # expand
relation_gene_phenotype["geneId"] = relation_gene_phenotype["geneId"].str.split(r",\s*")
relation_gene_phenotype["pid"] = relation_gene_phenotype["pid"].str.split(";").apply(lambda x: [s.strip() for s in x])
relation_gene_phenotype = relation_gene_phenotype.explode(["pid"]).reset_index(drop=True)
relation_gene_phenotype = relation_gene_phenotype.explode(["geneId"]).reset_index(drop=True)
relation_gene_phenotype = relation_gene_phenotype.drop_duplicates()
relation_gene_phenotype["geneId"] = pd.to_numeric(relation_gene_phenotype["geneId"]).astype('Int64')
relation_gene_phenotype['type'] = "Affect"

In [35]:
# relation_disease_phenotype (did, pid)
relation_disease_phenotype = GMDB[["OMIM", "present_features"]]
relation_disease_phenotype = relation_disease_phenotype.rename(columns={"OMIM": "did", "present_features":"pid", })
relation_disease_phenotype = relation_disease_phenotype.dropna(subset=['did', 'pid'])
    # expand
relation_disease_phenotype["pid"] = relation_disease_phenotype["pid"].str.split(";").apply(lambda x: [s.strip() for s in x])
relation_disease_phenotype = relation_disease_phenotype.explode(["pid"]).reset_index(drop=True)
relation_disease_phenotype = relation_disease_phenotype.drop_duplicates()
relation_disease_phenotype['did'] = pd.to_numeric(relation_disease_phenotype['did'], errors='coerce').astype('Int64')
relation_disease_phenotype['type'] = "Has_Phenotype"

In [36]:
# relation_sample_article (sid, pmid)
relation_sample_article = GMDB[["pmid"]]
relation_sample_article['sid'] = relation_sample_article['pmid']
relation_sample_article["sid"] = "s_" + relation_sample_article["sid"].astype(str) + "_" + (relation_sample_article.groupby("sid").cumcount() + 1).astype(str)
relation_sample_article = relation_sample_article.drop_duplicates()
relation_sample_article['type'] = "Exist"

C:\Users\15085\AppData\Local\Temp\ipykernel_26532\3799964966.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relation_sample_article['sid'] = relation_sample_article['pmid']
C:\Users\15085\AppData\Local\Temp\ipykernel_26532\3799964966.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relation_sample_article["sid"] = "s_" + relation_sample_article["sid"].astype(str) + "_" + (relation_sample_article.groupby("sid").cumcount() + 1).astype(str)


In [37]:
# relation_sample_disease_origin (sid, did) diagnosed_with
FGDD = pd.read_csv("FGDD.csv")
relation_sample_disease_origin = FGDD[["patient_id", "Disease_id"]]
relation_sample_disease_origin = relation_sample_disease_origin.rename(columns={"patient_id": "sid", "Disease_id":"did", })
relation_sample_disease_origin = relation_sample_disease_origin.dropna(subset=['sid', 'did'])
relation_sample_disease_origin['did'] = pd.to_numeric(relation_sample_disease_origin['did'], errors='coerce').astype('Int64')
relation_sample_disease_origin['type'] = "Diagnosed_With"

In [38]:
# relation_sample_disease (sid, did) diagnosed_with
relation_sample_disease = GMDB[["pmid", "OMIM"]]
relation_sample_disease = relation_sample_disease.rename(columns={"pmid": "sid", "OMIM":"did", })
relation_sample_disease["sid"] = "s_" + relation_sample_disease["sid"].astype(str) + "_" + (relation_sample_disease.groupby("sid").cumcount() + 1).astype(str)
relation_sample_disease = relation_sample_disease.drop_duplicates(subset=['sid', 'did'])
relation_sample_disease['did'] = pd.to_numeric(relation_sample_disease['did'], errors='coerce').astype('Int64')
relation_sample_disease['type'] = "Diagnosed_With"

In [40]:
relation_sample_phenotype.to_csv("./relationships/relation_sample_phenotype.csv" ,index=False)
relation_sample_variation.to_csv("./relationships/relation_sample_variation.csv" ,index=False)
relation_variation_gene.to_csv("./relationships/relation_variation_gene.csv" ,index=False)
relation_variation_disease.to_csv("./relationships/relation_variation_disease.csv" ,index=False)
relation_gene_phenotype.to_csv("./relationships/relation_gene_phenotype.csv" ,index=False)
relation_disease_phenotype.to_csv("./relationships/relation_disease_phenotype.csv" ,index=False)
relation_sample_article.to_csv("./relationships/relation_sample_article.csv" ,index=False)

relation_sample_disease_origin.to_csv("./relationships/relation_sample_disease_origin.csv" ,index=False)
relation_sample_disease.to_csv("./relationships/relation_sample_disease.csv" ,index=False)

# Cypher to import relationships
LOAD CSV WITH HEADERS FROM 'file:///relation_sample_phenotype.csv' AS row MATCH (n1:FacePhenotype {pid: row.pid}) MATCH (n2:Sample {sid: row.sid}) CREATE (n1)-[r: Mention_FP]->(n2) RETURN count(r);
LOAD CSV WITH HEADERS FROM 'file:///relation_sample_variation.csv' AS row MATCH (n1:Sample {sid: row.sid}) MATCH (n2:Variation {details: row.details}) CREATE (n1)-[r: Mention_Var]->(n2) RETURN count(r);
LOAD CSV WITH HEADERS FROM 'file:///relation_sample_article.csv' AS row MATCH (n1:Sample {sid: row.sid}) MATCH (n2:Article {pmid: toInteger(row.pmid)}) CREATE (n1)-[r: Exist]->(n2) RETURN count(r);

LOAD CSV WITH HEADERS FROM 'file:///relation_variation_gene.csv' AS row MATCH (n1:Variation {details: row.details}) MATCH (n2:Genotype {geneId: toInteger(row.geneId)}) CREATE (n1)-[r: Have]->(n2) RETURN count(r);
LOAD CSV WITH HEADERS FROM 'file:///relation_variation_disease.csv' AS row MATCH (n1:Variation {details: row.details}) MATCH (n2:Disease {did: toInteger(row.did)}) CREATE (n1)-[r: Cause]->(n2) RETURN count(r);

LOAD CSV WITH HEADERS FROM 'file:///relation_gene_phenotype.csv' AS row MATCH (n1:Genotype {geneId: toInteger(row.geneId)}) MATCH (n2:FacePhenotype {pid: row.pid}) CREATE (n1)-[r: Affect]->(n2) RETURN count(r);
LOAD CSV WITH HEADERS FROM 'file:///relation_disease_phenotype.csv' AS row MATCH (n1:Disease {did: toInteger(row.did)}) MATCH (n2:FacePhenotype {pid: row.pid}) CREATE (n1)-[r: Has_Phenotype]->(n2) RETURN count(r);



LOAD CSV WITH HEADERS FROM 'file:///relation_sample_disease_origin.csv' AS row MATCH (n1:Sample {sid: row.sid}) MATCH (n2:Disease {did: toInteger(row.did)}) CREATE (n1)-[r: Diagnosed_With]->(n2) RETURN count(r);
LOAD CSV WITH HEADERS FROM 'file:///relation_sample_disease.csv' AS row MATCH (n1:Sample {sid: row.sid}) MATCH (n2:Disease {did: toInteger(row.did)}) CREATE (n1)-[r: Diagnosed_With]->(n2) RETURN count(r);